```
Copyright 2021 IBM Corporation

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```

# Random Forest on Credit Card Fraud Dataset

## Background 

The goal of this learning task is to predict if a credit card transaction is fraudulent or genuine based on a set of anonymized features.

## Source

The raw dataset can be obtained directly from [Kaggle](https://www.kaggle.com/mlg-ulb/creditcardfraud). 

In this example, we download the dataset directly from Kaggle using their API. 

In order for this to work, you must login into Kaggle and folow [these instructions](https://www.kaggle.com/docs/api) to install your API token on your machine.


## Goal
The goal of this notebook is to illustrate how Snap ML can accelerate training of a random forest model on this dataset.

## Code

In [1]:
cd ../../

/home/aan/snapml-examples/examples


In [2]:
CACHE_DIR='cache-dir'

In [3]:
import numpy as np
import time
from datasets import CreditCardFraud
from sklearn.ensemble import RandomForestClassifier
from snapml import RandomForestClassifier as SnapRandomForestClassifier
from sklearn.metrics import roc_auc_score as score

In [4]:
dataset = CreditCardFraud(cache_dir=CACHE_DIR)
X_train, X_test, y_train, y_test = dataset.get_train_test_split()

Reading binary CreditCardFraud dataset (cache) from disk.


In [5]:
print("Number of examples: %d" % (X_train.shape[0]))
print("Number of features: %d" % (X_train.shape[1]))
print("Number of classes:  %d" % (len(np.unique(y_train))))

Number of examples: 213605
Number of features: 28
Number of classes:  2


In [6]:
# the dataset is highly imbalanced
labels, sizes = np.unique(y_train, return_counts=True)
print("%6.2f %% of the training transactions belong to class 0" % (sizes[0]*100.0/(sizes[0]+sizes[1])))
print("%6.2f %% of the training transactions belong to class 1" % (sizes[1]*100.0/(sizes[0]+sizes[1])))

from sklearn.utils.class_weight import compute_sample_weight
w_train = compute_sample_weight('balanced', y_train)
w_test = compute_sample_weight('balanced', y_test)

 99.83 % of the training transactions belong to class 0
  0.17 % of the training transactions belong to class 1


In [7]:
model = RandomForestClassifier(max_depth=8, n_estimators=100, n_jobs=4, random_state=42)
t0 = time.time()
model.fit(X_train, y_train, sample_weight=w_train)
t_fit_sklearn = time.time()-t0
score_sklearn = score(y_test, model.predict_proba(X_test)[:,1], sample_weight=w_test)
print("Training time (sklearn): %6.2f seconds" % (t_fit_sklearn))
print("ROC AUC score (sklearn): %.4f" % (score_sklearn))

Training time (sklearn):  18.18 seconds
ROC AUC score (sklearn): 0.9700


In [8]:
model = SnapRandomForestClassifier(max_depth=8, n_estimators=100, n_jobs=4, random_state=42, use_histograms=True)
t0 = time.time()
model.fit(X_train, y_train, sample_weight=w_train)
t_fit_snapml = time.time()-t0
score_snapml = score(y_test, model.predict_proba(X_test)[:,1], sample_weight=w_test)
print("Training time (snapml): %6.2f seconds" % (t_fit_snapml))
print("ROC AUC score (snapml): %.4f" % (score_snapml))

Training time (snapml):   2.20 seconds
ROC AUC score (snapml): 0.9790


In [9]:
speed_up = t_fit_sklearn/t_fit_snapml
score_diff = (score_snapml-score_sklearn)/score_sklearn
print("Speed-up:                %.1f x" % (speed_up))
print("Relative diff. in score: %.4f" % (score_diff))

Speed-up:                8.2 x
Relative diff. in score: 0.0092


## Disclaimer

Performance results always depend on the hardware and software environment. 

Information regarding the environment that was used to run this notebook are provided below:

In [10]:
import utils
environment = utils.get_environment()
for k,v in environment.items():
    print("%15s: %s" % (k, v))

       platform: Linux-4.15.0-136-generic-x86_64-with-glibc2.10
      cpu_count: 16
   cpu_freq_min: 1200.0
   cpu_freq_max: 3200.0
   total_memory: 62.825439453125
 snapml_version: 1.7.0
sklearn_version: 0.24.1


## Record Statistics

Finally, we record the enviroment and performance statistics for analysis outside of this standalone notebook.

In [11]:
import scrapbook as sb
sb.glue("result", {
    'dataset': dataset.name,
    'n_examples_train': X_train.shape[0],
    'n_examples_test': X_test.shape[0],
    'n_features': X_train.shape[1],
    'n_classes': len(np.unique(y_train)),
    'model': type(model).__name__,
    'score': score.__name__,
    't_fit_sklearn': t_fit_sklearn,
    'score_sklearn': score_sklearn,
    't_fit_snapml': t_fit_snapml,
    'score_snapml': score_snapml,
    'score_diff': score_diff,
    'speed_up': speed_up,
    **environment,
})